## Load Virtual Environment Python 3.10 !!

In [ ]:
import json
import pandas as pd
from radgraph import utils
import numpy as np 

# RadGraph - FULL COHORT

Original NER model based on radiology reports, requires >Python 3.8, <Python 3.10

Outputs dictionary with entities and relations described, broken down semantically to denote negation, severity etc. 



In [2]:
from radgraph import RadGraph
# initialize the radgraph instance  s
test_radgraph = RadGraph()
test_annotations = test_radgraph(["no evidence of acute cardiopulmonary process moderate hiatal hernia"])
print(test_annotations)

model_type not provided, defaulting to radgraph-xl
{'0': {'text': 'no evidence of acute cardiopulmonary process moderate hiatal hernia', 'entities': {'1': {'tokens': 'acute', 'label': 'Observation::definitely absent', 'start_ix': 3, 'end_ix': 3, 'relations': [['modify', '3']]}, '2': {'tokens': 'cardiopulmonary', 'label': 'Anatomy::definitely present', 'start_ix': 4, 'end_ix': 4, 'relations': []}, '3': {'tokens': 'process', 'label': 'Observation::definitely absent', 'start_ix': 5, 'end_ix': 5, 'relations': [['located_at', '2']]}, '4': {'tokens': 'moderate', 'label': 'Observation::definitely present', 'start_ix': 6, 'end_ix': 6, 'relations': [['modify', '5']]}, '5': {'tokens': 'hiatal hernia', 'label': 'Observation::definitely present', 'start_ix': 7, 'end_ix': 8, 'relations': []}}, 'data_source': None, 'data_split': 'inference'}}


In [3]:
# Load the structured report database and designate column for impression and findings (sections radgraph is designed to annotate) 
df_path = '../../../NLP_processing/raw_embeddings/full_cohort_pneumonia/cohort_structured_reports.csv'
database = pd.read_csv(df_path)
database.head()
database['radgraph_text'] = database['findings'] + database['impression']
database

,study_id,subject_id,report_path,full_text,examination,indication,technique,comparison,findings,impression,has_comparison,report_length,radgraph_text
0,51967283,10000980,../../../severity_data/report_files/p10/p10000...,FINAL REPORT INDICATION: [REMOVED]-year-old fe...,NaN,[REMOVED]-year-old female with shortness of br...,NaN,chest radiograph from [REMOVED] and [REMOVED]....,NaN,"right upper lobe pneumonia or mass. however, g...",True,797,NaN
1,54369281,10001884,../../../severity_data/report_files/p10/p10001...,FINAL REPORT EXAMINATION: Chest radiograph IND...,chest radiograph,dyspnea.,ap and lateral views of the chest.,[REMOVED],heart size is mildly enlarged. there is mild u...,no acute cardiopulmonary abnormality.,True,560,heart size is mildly enlarged. there is mild u...
2,58838312,10002428,../../../severity_data/report_files/p10/p10002...,FINAL REPORT PORTABLE CHEST: [REMOVED]. HISTOR...,NaN,NaN,NaN,NaN,single portable view of the chest is compared ...,no significant interval change with bilateral ...,False,761,single portable view of the chest is compared ...
3,52139270,10003502,../../../severity_data/report_files/p10/p10003...,FINAL REPORT HISTORY: Nausea and chllls. TECHN...,NaN,NaN,frontal and lateral views of the chest.,[REMOVED].,there are moderate bilateral pleural effusions...,"bilateral pleural effusions, cardiomegaly and ...",True,648,there are moderate bilateral pleural effusions...
4,57662923,10004322,../../../severity_data/report_files/p10/p10004...,WET READ: [REMOVED] [REMOVED] 2:03 PM Streaky ...,chest (pa and lat),history: [REMOVED]m with 2 weeks productive co...,pa and lateral views of the chest provided.,chest radiograph dated [REMOVED].,subtle streaky opacity in the left lower lobe ...,streaky left lobe opacity may reflect atelecta...,True,1345,subtle streaky opacity in the left lower lobe ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545,51053775,19941135,../../../severity_data/report_files/p19/p19941...,FINAL REPORT EXAMINATION: CHEST (PA AND LAT) I...,chest (pa and lat),history: [REMOVED]f with cough,chest pa and lateral,none.,cardiac silhouette size is normal. mediastinal...,focal ill-defined opacity in the left lung bas...,True,694,cardiac silhouette size is normal. mediastinal...
5546,52939041,19951239,../../../severity_data/report_files/p19/p19951...,WET READ: [REMOVED] [REMOVED] [REMOVED] 6:49 P...,NaN,history: [REMOVED]f with fever and cough // r/...,chest pa and lateral,none available,pa and lateral chest radiograph demonstrates a...,right upper lobe pneumonia.,True,585,pa and lateral chest radiograph demonstrates a...
5547,52809931,19963862,../../../severity_data/report_files/p19/p19963...,"FINAL REPORT HISTORY: Pneumonia for 9 days, ev...",NaN,NaN,frontal and lateral views of the chest.,none.,there is a consolidation in the right lower lo...,right lower lobe pneumonia. follow-up to resol...,True,673,there is a consolidation in the right lower lo...
5548,53101264,19992525,../../../severity_data/report_files/p19/p19992...,FINAL REPORT CHEST RADIOGRAPH PERFORMED ON [RE...,NaN,NaN,NaN,chest radiograph dated [REMOVED]. clinical his...,pa and lateral views of the chest were provide...,band-like opacity in the right lower lung coul...,True,792,pa and lateral views of the chest were provide...


In [4]:
import pandas as pd
from tqdm import tqdm

pneumonia_radgraph = RadGraph()  # Initialize RadGraph

# Ensure column is properly formatted
database["radgraph_text"] = database["radgraph_text"].astype(str).str.strip()  # Ensure strings

# Apply RadGraph to each report individually (with progress bar)
tqdm.pandas(desc="Processing RadGraph")
database["extracts"] = database["radgraph_text"].progress_apply(pneumonia_radgraph)

# Verify results
print(database["extracts"].head())

model_type not provided, defaulting to radgraph-xl


Processing RadGraph:   0%|          | 0/5550 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable /Users/JanayeCheong/Documents/DATATHON/nusdatathon2024/.venv_310/lib/python3.10/site-packages/radgraph/dygie/data/dataset_readers/dygie.py:192: UserWarning: Document 0 has a sentence with a single token or no tokens. This may break the modeling code.
  warnings.warn(msg)
TOKENIZERS_PARALLELISM=(true | false)
Processing RadGraph: 100%|██████████| 5550/5550 [09:02<00:00, 10.24it/s]

0    {'0': {'text': 'nan', 'entities': {}, 'data_so...
1    {'0': {'text': 'heart size is mildly enlarged ...
2    {'0': {'text': 'single portable view of the ch...
3    {'0': {'text': 'there are moderate bilateral p...
4    {'0': {'text': 'subtle streaky opacity in the ...
Name: extracts, dtype: object


In [5]:
database.to_csv('cohort_ner_database.csv')

In [6]:
import torch
torch.save(database, 'cohort_ner_database.pt')

In [7]:
ner_database = pd.read_csv('cohort_ner_database.csv')

In [8]:
ner_database.shape

(5550, 15)